In [1]:
import networkx as nx
import dwave_networkx as dnx
import random
import matplotlib.pyplot as plt
import pulp
import numpy as np

from lpSolver import solve
from uccgGenerator import tree_insertion, Graph
from plotNetwork import plotGraph, plotCoupling

In [2]:
delta = 3
num_colors = 2 * delta + 1

In [3]:
def get_cmap(n, name='hsv'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)

In [4]:
cmap = get_cmap(num_colors)
colors= {cmap(i) for i in range(num_colors)}

In [ ]:
def is_valid(g):
    for u, v in g.edges:
        if g.nodes[u]['color'] == g.nodes[v]['color']:
            return False
    return True

In [ ]:
def get_available_colors(g, u):
    used_colors = [g.nodes[v]['color'] for v in g.neighbors(u)]
    available = colors - set(used_colors)
    return available

In [ ]:
def markov_move(g):
    cg = nx.Graph(g)
    u = random.sample(cg.nodes, 1)[0]
    c = random.sample(get_available_colors(cg, u), 1)[0]
    cg.nodes[u]['color'] = c
    return cg

In [ ]:
def get_children(g):
    children = []
    prob = 1/(len(g)*len(colors))
    total = 0
    for u in g.nodes:
        for c in get_available_colors(g, u):
            cg = nx.Graph(g)
            cg.nodes[u]['color'] = c
            cg.prob = prob
            children.append(cg)
            total += prob
    g_stay = nx.Graph(g)
    g_stay.prob = 1 - total
    return children + [g_stay]

In [ ]:
def get_dist(g1, g2):
    count = 0
    for u in g1.nodes:
        if g1.nodes[u]['color'] != g2.nodes[u]['color']:
            count += 1
    return count

In [ ]:
g1 = nx.random_regular_graph(delta, 12)
for u in g1.nodes:
    g1.nodes[u]['color'] = random.sample(colors, 1)[0]

In [ ]:
while not is_valid(g1):
    g1 = markov_move(g1)

In [ ]:
g2 = random.choice(get_children(g1))
while get_dist(g1, g2) < 1:
    g2 = random.choice(get_children(g1))

g1_children = get_children(g1)
g2_children = get_children(g2)

l1, l2 = len(g1_children), len(g2_children)

dists = np.zeros((l1, l2))
for i in range(l1):
    for j in range(l2):
        dists[i][j] = get_dist(g1_children[i], g2_children[j])

status, dist, joint_prob = solve(g1_children, g2_children, dists)
print(dist)